In [1]:
import xml.etree.cElementTree as et
import pandas as pd
import random

/nfs/2017/s/ssong/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/nfs/2017/s/ssong/.pyenv/versions/3.6.5/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
path = 'data/train.xml'
dfcols = ['category', 'question_id', 'title']

root = et.parse(path)
rows = root.findall('.//question')

# NESTED LIST
xml_data = [[row.get('category'), row.get('question_id'), row.get('title')] 
            for row in rows]

df_xml = pd.DataFrame(xml_data, columns=dfcols)

#shuffle dataframe
df_xml = df_xml.sample(frac=1).reset_index(drop=True)
df_xml

,category,question_id,title
0,unix,195350,Heirloom-mailx and Saving to IMAP Folder with ...
1,unix,213233,rsync: how to include files with name ending i...
2,askubuntu.com,103667,x64 or x32 best for virtualbox to install wind...
3,askubuntu.com,107702,Guake Doesn't Start When Session Starts
4,softwareengineering,155043,"If an ""either/or"" algorithm is ""Boolean"", what..."
5,devops,61747,How efficiently does GlusterFS deal with failu...
6,askubuntu.com,117151,bitcoin miner compatible with nvidia
7,unix,185075,How to join rows with single columns to a maxi...
8,softwareengineering,160466,Is objected oriented programming paradigm outd...
9,opensource,59620,Can I use JavaFX in a Commercial Application f...


In [3]:
df_xml['category'].value_counts()

askubuntu.com          67037
unix                   41829
softwareengineering    25090
codereview             18098
dba                    17117
gamedev                12517
raspberrypi             5766
bitcoin                 5107
arduino                 4222
crypto                  3934
ethereum                3549
emacs                   2864
vi                      1462
robotics                1256
opendata                 998
opensource               625
ai                       500
devops                   413
coffee                   413
beer                     402
computergraphics         370
bioinformatics           361
iot                      258
Name: category, dtype: int64

In [4]:
AllQuestions = df_xml['title'].tolist()

In [5]:
#Dependencies to convert to embeddings
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /nfs/2017/s/ssong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
#fuck yes I have all questions in a list
print(AllQuestions[:5])

['Heirloom-mailx and Saving to IMAP Folder with Spaces', 'rsync: how to include files with name ending in a specific string', 'x64 or x32 best for virtualbox to install windows 8 or 7', "Guake Doesn't Start When Session Starts", 'If an "either/or" algorithm is "Boolean", what\'s a "zero, one, or many" algorithm?']


In [7]:
#Now just got to tokenize it and then set tags for each questions
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(AllQuestions)]

In [ ]:
#Use Doc2Vec to create numerical representation of document as well as word meanings.

max_epochs = 10000
vec_size = 100
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

/nfs/2017/s/ssong/.pyenv/versions/3.6.5/envs/jupyter/lib/python3.6/site-packages/gensim/models/doc2vec.py:535: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


iteration 0


/nfs/2017/s/ssong/.pyenv/versions/3.6.5/envs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
